In [182]:
import pandas as pd
from catboost import CatBoostClassifier
from catboost import Pool
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

In [183]:
X = pd.read_csv('train_data.csv',index_col='ID')
X_test = pd.read_csv('test_data.csv',index_col='ID')


In [184]:
X.head()

,parents,has_nurs,form,children,housing,finance,social,health,app_status
ID,,,,,,,,,
1,usual,less_proper,complete,3,critical,convenient,problematic,not_recom,0
2,pretentious,very_crit,completed,1,convenient,inconv,nonprob,not_recom,0
3,pretentious,proper,incomplete,1,less_conv,convenient,slightly_prob,priority,1
4,great_pret,improper,complete,1,convenient,convenient,nonprob,recommended,1
5,great_pret,less_proper,completed,1,convenient,convenient,slightly_prob,priority,1


In [185]:
y = X.app_status
X.drop(['app_status'],axis=1,inplace=True)

In [186]:
X.head()

,parents,has_nurs,form,children,housing,finance,social,health
ID,,,,,,,,
1,usual,less_proper,complete,3,critical,convenient,problematic,not_recom
2,pretentious,very_crit,completed,1,convenient,inconv,nonprob,not_recom
3,pretentious,proper,incomplete,1,less_conv,convenient,slightly_prob,priority
4,great_pret,improper,complete,1,convenient,convenient,nonprob,recommended
5,great_pret,less_proper,completed,1,convenient,convenient,slightly_prob,priority


In [187]:
X_train,X_val,y_train,y_val = train_test_split(X,y,train_size=0.8,random_state=0)

In [188]:
[cname for cname in X_train.columns if X_train[cname].dtype == 'object']

['parents',
 'has_nurs',
 'form',
 'children',
 'housing',
 'finance',
 'social',
 'health']

In [189]:
# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)
X_test = pd.get_dummies(X_test)
X_train, X_val = X_train.align(X_val, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [190]:
X_train.head()

,parents_great_pret,parents_pretentious,parents_usual,has_nurs_critical,has_nurs_improper,has_nurs_less_proper,has_nurs_proper,has_nurs_very_crit,form_complete,form_completed,...,housing_critical,housing_less_conv,finance_convenient,finance_inconv,social_nonprob,social_problematic,social_slightly_prob,health_not_recom,health_priority,health_recommended
ID,,,,,,,,,,,,,,,,,,,,,
5314,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
7741,0,1,0,0,0,0,1,0,0,1,...,1,0,1,0,0,0,1,0,1,0
1333,1,0,0,1,0,0,0,0,1,0,...,0,1,0,1,1,0,0,0,0,1
4520,0,0,1,0,0,0,1,0,1,0,...,0,1,1,0,0,0,1,1,0,0
8579,0,1,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,1,0


In [258]:
#xgboost

model = XGBClassifier(n_estimators=50, learning_rate=0.05 )
#model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_train,y_train)
predictions = model.predict(X_val)
#model = CatBoostClassifier(iterations=1000,learning_rate=0.01)
#depth=1,random_state=3)
# train_pool=Pool(data=X_train,label=y_train)
# val_pool=Pool(data=X_val,label=y_val)
# model.fit(train_pool)
# predictions = model.predict(val_pool)





c:\Python37\cv\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[02:11:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [259]:
from sklearn.metrics import classification_report,confusion_matrix


print(confusion_matrix(y_val,predictions))
print(classification_report(y_val,predictions))

[[1320   13]
 [  39  702]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1333
           1       0.98      0.95      0.96       741

    accuracy                           0.97      2074
   macro avg       0.98      0.97      0.97      2074
weighted avg       0.98      0.97      0.97      2074



In [260]:
from sklearn.metrics import f1_score,auc,precision_recall_curve
f1_score(y_val,predictions,average='macro')

0.9724846104860962

In [261]:
precision,recall,_ = precision_recall_curve(y_val,predictions)
auc_score = auc(recall,precision)
print(auc_score)

0.9739954229397462


In [263]:
preds_test = model.predict(X_test)
output = pd.DataFrame({'ID': X_test.index,'app_status': preds_test})
output.to_csv('Noob Hackers.csv',index=False)
